In [1]:
import os
transcript_dir = r"..\transcripts"

transcripts = {}
for filename in os.listdir(transcript_dir):
    with open(os.path.join(transcript_dir, filename), 'r') as f:
        transcripts[filename.split(".")[0]] = f.read()

transcripts.keys()

dict_keys(['4tools', 'baselifesciences', 'bookstar', 'calmsea', 'deel', 'explorer', 'keeper', 'lasebane', 'mode', 'ss', 'stackcentral', 'xp'])

In [2]:
labels = {
    "4tools": """Priority: “Understand content workflow with multiple contractors so you can increase your engagement size from 10k to 20k" (see 3:07 and 8:41)
Event: “Trial expires in 8 days” (see 3:07 and 8:41)
Date: <2 weeks from now> (see 43:18)
Message: Hi Ryan, last we spoke, it was important to understand your content workflow across clients so you can scale up your engagement size to 20k. Your trial has ended. Wanted to reconnect to see if you’re ready to make a decision to proceed or not.""",
    
    "bookstar": """Priority: “Traditional outreach is failing and needs to invest in social selling but marketing is stretched thin" (see 2:40)
Event: “Kate to talk to her boss” (see 31:24)
Date: <next June> (see 38:26)
Message: Hi Kate, last we spoke, it was important to understand your content workflow across clients so you can scale up your engagement size to 20k. Your trial expires tomorrow. Wanted to reconnect to see if you’re ready to make a decision to proceed or not.""",
    
    "xp": """Priority: “Get rid of Terraform Cloud in Q3 because support is not great" (see 3:42)
Event: “TCF contracts expires” (see 31:24)
Date: <next December> (see 5:03)
Message: Hi Mike, last we spoke, it was important to move off Terraform Cloud in Q3. I know your contract ends in December. Wanted to reconnect to see if we’re on track to replace TFC.""",
    
    "mode": """Priority: “Increase thought leadership as part of ABM execution" (see 22:42)
Event: “ABM research is complete” (see 9:53)
Date: <next December> (see 5:03)
Message: Hi Swati, hope your conference in Copenhagen went well. When we last spoke, you were wrapping up the ABM account research phase. You mentioned you’d be able to enter the execution phase. Wanted to reconnect to see if you’re ready to improve Jan’s LinkedIn presence to reach key decision makers in November.""",

    }

In [3]:
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [5]:
from crewai import Agent, Task, Crew, Process, LLM

llm = LLM(
    model = "azure/gpt-4o",
    # api_key=os.environ["OPENAI_API_KEY"],
    # base_url=os.environ["AZURE_OPENAI_ENDPOINT"],
)

In [9]:
call_analyzer = Agent(
  role='Senior Sales Analyst',
  goal='Uncover the hidden action items in sales calls',
  backstory="""You work as a sales person in a startup.
Your expertise lies in identifying hidden action items form the sales call.
You identify things that are not explicitly mentioned but are important next steps to the sales process.""",
  verbose=True,
  allow_delegation=False,
  llm=llm,
#   tools=[]
)
writer = Agent(
  role='Sales Messaging Specialist',
  goal='Craft a crisp message reminding the client of the action items',
  backstory="""You are a sales messaging specialist. Your expertise lies in crafting messages that are clear, concise and actionable basd on the hidden action items identified by the sales analyst.
  Only provide the action items that are more than 2 weeks away.""",
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

# Create tasks for your agents
task1 = Task(
  description="""Analyze the sales call transcript and provide hidden action items, if any.
Find the action items and when to reach out to the client. Only the action items that are more than 2 weeks away are considered.

Here is the response format:
<format>
Priority: <action item in one line> (timestamp in the call, if available)
Event: <event in one line> (timestamp in the call, if available)
Date: <when to reach out to the client> (timestamp in the call, if available)
</format>

Here is a sample response:
<sample>
Priority: "Increase thought leadership as part of ABM execution" (see 22:42)
Event: "ABM research is complete" (see 9:53)
Date: <next December> (see 5:03)
</sample>

Here is the sales call transcript:
<trascript>
{sales_call_transcript}
</transcript>""",

  expected_output="""List the priority, event and date to reach out to the client in the format provided.
<sample>
Priority: "Increase thought leadership as part of ABM execution" (see 22:42)
Event: "ABM research is complete" (see 9:53)
Date: <next December> (see 5:03)
</sample>
XML tags should not be included in the response.
""",

  agent=call_analyzer
)

task2 = Task(
  description="""Using the hidden action items, craft a message to remind the client of the action items and provide the date to reach out to the client.

Here is a sample:
<sample_item>
Priority: "Increase thought leadership as part of ABM execution" (see 22:42)
Event: "ABM research is complete" (see 9:53)
Date: <next December> (see 5:03)
</sample_item>

<sample_response>
Message: Hi Swati, hope your conference in Copenhagen went well. When we last spoke, you were wrapping up the ABM account research phase. You mentioned you’d be able to enter the execution phase. Wanted to reconnect to see if you’re ready to improve Jan’s LinkedIn presence to reach key decision makers in November.
</sample_response>

""",

  expected_output="""A message for each action item to remind the client of the action items and the date to reach out to the client.
Here is the response format:
<format>
Message: <message to remind the client of the action items and the date to reach out to the client.>
</format>
XML tags should not be included in the response.
""",
  agent=writer
)

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[call_analyzer, writer],
  tasks=[task1, task2],
  verbose=True,
  # process = Process.sequential
)


2024-09-26 11:43:31,832 - 14460 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


Message: Hi [Client Name], hope you're doing well. Just a reminder of our upcoming action item: We need to evaluate Letterdrop's agency-specific features and how they might fit into your workflow. Taylor is set to invite Marc and Sean to their existing workspace for further testing in two weeks. Best, [Your Name]


In [ ]:
inputs = {
    "sales_call_transcript": transcripts["4tools"]
}

result = crew.kickoff(inputs=inputs)

In [11]:
print(result.tasks_output[0].raw)
print(result.tasks_output[1].raw)

Priority: "Explore agency-specific functionalities and needs" (see 45:01)
Event: "Discussion on agency-side features and processes" (see 46:46)
Date: <next evaluation period, roughly in 2 months> (see 25:31)
Message: Hi [Client's Name], I hope you’re doing well. When we last spoke, we discussed the functionalities and needs specific to agency-side features. You mentioned you’d be open to further exploration during the next evaluation period, which is roughly in two months. Just wanted to remind you about this and check if you’re ready to revisit this discussion. Looking forward to your thoughts!


In [ ]:
call_analyzer.verbose = False
writer.verbose = False

res = {}

for k,v in transcripts.items():
    inputs = {
        "sales_call_transcript": v
    }

    result = crew.kickoff(inputs=inputs)
    res[k] = result

I had asked model to find "items" ie mulitple if available, and for each task there is a message. 

In [17]:
for k, v in res.items():
    print(f"{k}: ")
    print(result.tasks_output[0].raw)
    print(result.tasks_output[1].raw)
    print("\n-----------------------------------\n")

4tools: 
Priority: "Evaluate POC success criteria and set up sessions" (see 01:01)
Event: "POC success criteria reviewed with teams" (see 01:26)
Date: <next December> (see 05:03)

Priority: "Ensure the right permissions for cloud and VCS integrations" (see 02:15)
Event: "Permissions for integrations verified" (see 04:44)
Date: <next December> (see 05:03)

Priority: "Review and approve NDA documents with legal team" (see 08:20)
Event: "Legal team approval of NDA" (see 08:54)
Date: <next December> (see 05:03)
Message: Hi [Client's Name], hope you're doing well. When we last spoke, you were reviewing the POC success criteria with your teams. You mentioned you'd be ready to set up the sessions soon. Let's reconnect in December to evaluate the success criteria and plan the next steps.

Message: Hi [Client's Name], I hope everything is progressing smoothly on your end. Last time, you were verifying the permissions for cloud and VCS integrations. You mentioned that everything would be ready b